# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 [Online.](https://arxiv.org/abs/1611.06455 "Wang et al. (2017)")

## Data
Additional datasets are available at http://www.timeseriesclassification.com/. Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

In [3]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Train MLP
Expected running time for various datasets, on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset | Training time
:-------| :--------
Adiac   | 18 minutes


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os
import time

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

fdir = '../data'  
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['Adiac']
for each in flist:
    fname = each
    print('Running dataset', fname)
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = min(x_train.shape[0]/10, 16)
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8)    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    end = time.time()
    print('Training time (in seconds)', end-start)
    #Print the testing results which has the lowest training loss.
    print('Test result with the lowest training loss. Loss and accuracy are -')
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

 



Running dataset Adiac
Train on 390 samples, validate on 391 samples
Epoch 1/5000
390/390 [==============================] - 1s 1ms/step - loss: 3.6910 - acc: 0.0333 - val_loss: 3.6301 - val_acc: 0.0716
Epoch 2/5000
390/390 [==============================] - 0s 540us/step - loss: 3.5710 - acc: 0.0590 - val_loss: 3.5107 - val_acc: 0.0921
Epoch 3/5000
390/390 [==============================] - 0s 545us/step - loss: 3.4638 - acc: 0.0872 - val_loss: 3.4549 - val_acc: 0.0972
Epoch 4/5000
390/390 [==============================] - 0s 562us/step - loss: 3.4003 - acc: 0.0923 - val_loss: 3.3509 - val_acc: 0.0639
Epoch 5/5000
390/390 [==============================] - 0s 558us/step - loss: 3.3128 - acc: 0.1077 - val_loss: 3.3520 - val_acc: 0.1100
Epoch 6/5000
390/390 [==============================] - 0s 554us/step - loss: 3.2449 - acc: 0.1410 - val_loss: 3.2656 - val_acc: 0.1355
Epoch 7/5000
390/390 [==============================] - 0s 544us/step - loss: 3.2070 - acc: 0.1487 - val_loss: 3.2202 

390/390 [==============================] - 0s 545us/step - loss: 1.6850 - acc: 0.4308 - val_loss: 1.7456 - val_acc: 0.4246
Epoch 61/5000
390/390 [==============================] - 0s 552us/step - loss: 1.6395 - acc: 0.4564 - val_loss: 2.0632 - val_acc: 0.3734
Epoch 62/5000
390/390 [==============================] - 0s 571us/step - loss: 1.6335 - acc: 0.4949 - val_loss: 1.6273 - val_acc: 0.4731
Epoch 63/5000
390/390 [==============================] - 0s 559us/step - loss: 1.6572 - acc: 0.4487 - val_loss: 1.8995 - val_acc: 0.3862
Epoch 64/5000
390/390 [==============================] - 0s 559us/step - loss: 1.5580 - acc: 0.4923 - val_loss: 1.8195 - val_acc: 0.3913
Epoch 65/5000
390/390 [==============================] - 0s 551us/step - loss: 1.5199 - acc: 0.4974 - val_loss: 1.6290 - val_acc: 0.4783
Epoch 66/5000
390/390 [==============================] - 0s 552us/step - loss: 1.5607 - acc: 0.4821 - val_loss: 1.7172 - val_acc: 0.4220
Epoch 67/5000
390/390 [==============================] 

Epoch 120/5000
390/390 [==============================] - 0s 537us/step - loss: 1.1720 - acc: 0.5615 - val_loss: 1.3863 - val_acc: 0.6036
Epoch 121/5000
390/390 [==============================] - 0s 578us/step - loss: 1.1260 - acc: 0.5923 - val_loss: 1.5517 - val_acc: 0.5269
Epoch 122/5000
390/390 [==============================] - 0s 538us/step - loss: 1.1179 - acc: 0.6000 - val_loss: 1.3655 - val_acc: 0.5652
Epoch 123/5000
390/390 [==============================] - 0s 531us/step - loss: 1.0820 - acc: 0.6231 - val_loss: 1.4098 - val_acc: 0.5754
Epoch 124/5000
390/390 [==============================] - 0s 530us/step - loss: 1.1076 - acc: 0.6154 - val_loss: 1.6969 - val_acc: 0.4834
Epoch 125/5000
390/390 [==============================] - 0s 543us/step - loss: 1.1902 - acc: 0.5692 - val_loss: 1.3894 - val_acc: 0.5678
Epoch 126/5000
390/390 [==============================] - 0s 542us/step - loss: 1.0826 - acc: 0.6359 - val_loss: 1.5537 - val_acc: 0.5038
Epoch 127/5000
390/390 [==========

390/390 [==============================] - 0s 542us/step - loss: 0.9508 - acc: 0.6667 - val_loss: 1.3034 - val_acc: 0.6266
Epoch 180/5000
390/390 [==============================] - 0s 557us/step - loss: 0.9053 - acc: 0.6692 - val_loss: 1.5321 - val_acc: 0.5499
Epoch 181/5000
390/390 [==============================] - 0s 558us/step - loss: 0.8730 - acc: 0.6846 - val_loss: 1.4716 - val_acc: 0.5959
Epoch 182/5000
390/390 [==============================] - 0s 546us/step - loss: 0.8953 - acc: 0.6923 - val_loss: 1.8983 - val_acc: 0.4552
Epoch 183/5000
390/390 [==============================] - 0s 536us/step - loss: 0.8490 - acc: 0.7000 - val_loss: 1.3095 - val_acc: 0.6394
Epoch 184/5000
390/390 [==============================] - 0s 543us/step - loss: 0.9657 - acc: 0.6333 - val_loss: 1.8450 - val_acc: 0.4783
Epoch 185/5000
390/390 [==============================] - 0s 543us/step - loss: 0.8572 - acc: 0.7077 - val_loss: 1.6238 - val_acc: 0.5601
Epoch 186/5000
390/390 [=========================

390/390 [==============================] - 0s 551us/step - loss: 0.7745 - acc: 0.7231 - val_loss: 2.6698 - val_acc: 0.4220
Epoch 239/5000
390/390 [==============================] - 0s 534us/step - loss: 0.8039 - acc: 0.6949 - val_loss: 1.7937 - val_acc: 0.5090
Epoch 240/5000
390/390 [==============================] - 0s 539us/step - loss: 0.7785 - acc: 0.7000 - val_loss: 1.6455 - val_acc: 0.5396
Epoch 241/5000
390/390 [==============================] - 0s 532us/step - loss: 0.8458 - acc: 0.6872 - val_loss: 1.3569 - val_acc: 0.6368
Epoch 242/5000
390/390 [==============================] - 0s 532us/step - loss: 0.6694 - acc: 0.7487 - val_loss: 1.8675 - val_acc: 0.5192
Epoch 243/5000
390/390 [==============================] - 0s 542us/step - loss: 0.7953 - acc: 0.7154 - val_loss: 1.3125 - val_acc: 0.6445
Epoch 244/5000
390/390 [==============================] - 0s 549us/step - loss: 0.7950 - acc: 0.7231 - val_loss: 2.1195 - val_acc: 0.4552
Epoch 245/5000
390/390 [=========================

...

390/390 [==============================] - 0s 531us/step - loss: 0.3199 - acc: 0.8667 - val_loss: 1.9085 - val_acc: 0.6445
Epoch 829/5000
390/390 [==============================] - 0s 536us/step - loss: 0.4121 - acc: 0.8359 - val_loss: 2.2496 - val_acc: 0.6240
Epoch 830/5000
390/390 [==============================] - 0s 529us/step - loss: 0.3250 - acc: 0.8846 - val_loss: 1.6654 - val_acc: 0.6905
Epoch 831/5000
390/390 [==============================] - 0s 537us/step - loss: 0.2701 - acc: 0.9000 - val_loss: 1.7545 - val_acc: 0.6726
Epoch 832/5000
390/390 [==============================] - 0s 532us/step - loss: 0.2683 - acc: 0.8974 - val_loss: 1.8604 - val_acc: 0.6752
Epoch 833/5000
390/390 [==============================] - 0s 533us/step - loss: 0.4378 - acc: 0.8667 - val_loss: 1.9611 - val_acc: 0.6522
Epoch 834/5000
390/390 [==============================] - 0s 540us/step - loss: 0.2763 - acc: 0.8795 - val_loss: 1.8548 - val_acc: 0.6854
Epoch 835/5000
390/390 [=========================

390/390 [==============================] - 0s 544us/step - loss: 0.2840 - acc: 0.9051 - val_loss: 2.1604 - val_acc: 0.6368
Epoch 888/5000
390/390 [==============================] - 0s 539us/step - loss: 0.3548 - acc: 0.8846 - val_loss: 1.9553 - val_acc: 0.6624
Epoch 889/5000
390/390 [==============================] - 0s 537us/step - loss: 0.3245 - acc: 0.8744 - val_loss: 1.7446 - val_acc: 0.7263
Epoch 890/5000
390/390 [==============================] - 0s 535us/step - loss: 0.2472 - acc: 0.9128 - val_loss: 1.7581 - val_acc: 0.6496
Epoch 891/5000
390/390 [==============================] - 0s 530us/step - loss: 0.3007 - acc: 0.9000 - val_loss: 1.7967 - val_acc: 0.6726
Epoch 892/5000
390/390 [==============================] - 0s 536us/step - loss: 0.2251 - acc: 0.9179 - val_loss: 2.2110 - val_acc: 0.6496
Epoch 893/5000
390/390 [==============================] - 0s 532us/step - loss: 0.2730 - acc: 0.9077 - val_loss: 2.2318 - val_acc: 0.6292
Epoch 894/5000
390/390 [=========================

390/390 [==============================] - 0s 529us/step - loss: 0.3784 - acc: 0.8564 - val_loss: 1.7013 - val_acc: 0.7059
Epoch 947/5000
390/390 [==============================] - 0s 539us/step - loss: 0.2472 - acc: 0.8949 - val_loss: 1.6737 - val_acc: 0.7110
Epoch 948/5000
390/390 [==============================] - 0s 533us/step - loss: 0.2613 - acc: 0.9128 - val_loss: 1.7311 - val_acc: 0.7110
Epoch 949/5000
390/390 [==============================] - 0s 540us/step - loss: 0.3842 - acc: 0.8641 - val_loss: 1.7256 - val_acc: 0.7008
Epoch 950/5000
390/390 [==============================] - 0s 541us/step - loss: 0.2032 - acc: 0.9282 - val_loss: 1.7756 - val_acc: 0.6982
Epoch 951/5000
390/390 [==============================] - 0s 540us/step - loss: 0.1928 - acc: 0.9205 - val_loss: 1.7837 - val_acc: 0.6803
Epoch 952/5000
390/390 [==============================] - 0s 535us/step - loss: 0.4058 - acc: 0.8718 - val_loss: 1.6076 - val_acc: 0.7161
Epoch 953/5000
390/390 [=========================

390/390 [==============================] - 0s 531us/step - loss: 0.2290 - acc: 0.9128 - val_loss: 1.7652 - val_acc: 0.7059
Epoch 1006/5000
390/390 [==============================] - 0s 540us/step - loss: 0.3978 - acc: 0.8744 - val_loss: 2.5968 - val_acc: 0.5371
Epoch 1007/5000
390/390 [==============================] - 0s 538us/step - loss: 0.2230 - acc: 0.9256 - val_loss: 1.8213 - val_acc: 0.6982
Epoch 1008/5000
390/390 [==============================] - 0s 538us/step - loss: 0.1758 - acc: 0.9385 - val_loss: 1.7855 - val_acc: 0.6880
Epoch 1009/5000
390/390 [==============================] - 0s 543us/step - loss: 0.2033 - acc: 0.9282 - val_loss: 1.8748 - val_acc: 0.6701
Epoch 1010/5000
390/390 [==============================] - 0s 539us/step - loss: 0.2983 - acc: 0.8795 - val_loss: 2.0326 - val_acc: 0.6701
Epoch 1011/5000
390/390 [==============================] - 0s 538us/step - loss: 0.2872 - acc: 0.8897 - val_loss: 2.2467 - val_acc: 0.6036
Epoch 1012/5000
390/390 [==================

390/390 [==============================] - 0s 541us/step - loss: 0.2412 - acc: 0.9128 - val_loss: 1.6955 - val_acc: 0.7212
Epoch 1065/5000
390/390 [==============================] - 0s 532us/step - loss: 0.2783 - acc: 0.8897 - val_loss: 1.9125 - val_acc: 0.6752
Epoch 1066/5000
390/390 [==============================] - 0s 534us/step - loss: 0.3532 - acc: 0.8821 - val_loss: 1.9731 - val_acc: 0.6394
Epoch 1067/5000
390/390 [==============================] - 0s 524us/step - loss: 0.2347 - acc: 0.9154 - val_loss: 2.0278 - val_acc: 0.6675
Epoch 1068/5000
390/390 [==============================] - 0s 545us/step - loss: 0.1970 - acc: 0.9154 - val_loss: 2.1038 - val_acc: 0.6471
Epoch 1069/5000
390/390 [==============================] - 0s 548us/step - loss: 0.3676 - acc: 0.8615 - val_loss: 1.8049 - val_acc: 0.6931
Epoch 1070/5000
390/390 [==============================] - 0s 538us/step - loss: 0.1449 - acc: 0.9487 - val_loss: 2.0904 - val_acc: 0.6445
Epoch 1071/5000
272/390 [==================

# Recorded results
## Adiac

|Run |Loss |Accuracy
|:---|:--- | :---
|1   |0.005222544357037315  | 0.6930946294608933
|2   |0.0018498263113997382 |0.7109974427601261